# **Linguagem Kobra**

## Desenvolvedores
- [João Lucas de Moraes Barros Cadorniga](https://github.com/JoaoLucasMBC)
- [Eduardo Mendes Vaz](https://github.com/EduardoMVAz)

## Funcionamento e Documentação

todo: gramática

Nesse repositório está o código para a linguagem Kobra. Para utilizar a linguagem, clone o repositório na sua máquina:
        git clone https://github.com/JoaoLucasMBC/paradigmas-kobra.git

Crie um ambiente virtual na pasta do projeto:

    windows:
        python -m venv env
    linux:
        python3 -m venv env

E instale todas as dependências do projeto, presentes em `requirements.txt`:

        pip install -r requirements.txt

No notebook `kobra.ipynb`, está a linguagem está dividida em paradigmas demonstrando o processo de desenvolvimento, com exemplos.

Já o arquivo `kobra.py` é como um "compilador" da linguagem. Você pode usar esse arquivo para rodar códigos escritos em Kobra. Para fazer isso, você deve criar um arquivo de texto na pasta do repositório, **que contenha um código Kobra válido** (para entender como usar a linguagem, consulte a gramática acima ou um dos exemplos `.kbr` presentes no repositório) e então executar o seguinte comando:

    windows:
        python kobra.py nome-do-arquivo-aqui.extensão-do-arquivo
    linux:
        python3 kobra.py nome-do-arquivo-aqui.extensão-do-arquivo

O executor irá então imprimir no terminal o resultado da execução do arquivo. Para o exemplo `in02.kbr`, a saída no terminal seria:

![in02](assets/kobra.png)

Sendo que os prints em abaixo de "RUNTIME" representam as coisas impressas durante a execução do código Kobra, e o "Result" representa o **estado final das variáveis criadas no código**.

## Dependências e Funcionalidades

A linguagem Kobra foi desenvolvida usando Python como base e usando a biblioteca rply.

In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('MAIN', r'main')

lg.add('NUMBER', r'\d+(.\d+)?')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('POW', r'\^')
lg.add('PRINT', r'print')
lg.add('RETURN', r'return')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('OPEN_CURLY', r'{')
lg.add('CLOSE_CURLY', r'}')
lg.add('OPEN_SQUARE', r'\[')
lg.add('CLOSE_SQUARE', r'\]')

lg.add('COLON', r':')

lg.add('VOID', r'void')
lg.add('INT', r'int')
lg.add('FLOAT', r'float')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')

lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.add('ENDLINE', r';')

lg.add('STRING', r'\".*\"')

lg.ignore('\s+')
lg.ignore('//.*\n')
lg.ignore('\t+')
lg.ignore('\n+')

lexer = lg.build()

In [3]:
from rply.token import BaseBox

class Main(BaseBox):
    def __init__(self, vars, instrs, funcs):
        self.vars = vars
        self.instrs = instrs
        self.funcs = funcs

    def accept(self, visitor):
        visitor.visit_main(self)

## FUNCTIONS

class Funcs(BaseBox):
    def __init__(self, func, funcs):
        self.func = func
        self.funcs = funcs

    def accept(self, visitor):
        visitor.visit_funcs(self)

class Func(BaseBox):
    def __init__(self, ret_type, id, arg, vars, instrs, ret):
       self.ret_type = ret_type
       self.id = id
       self.arg = arg
       self.vars = vars
       self.instrs = instrs
       self.ret = ret

    def accept(self, visitor):
        visitor.visit_func(self)

## RESTO


class Vars(BaseBox):
    def __init__(self, var, vars):
        self.var = var
        self.vars = vars

    def accept(self, visitor):
        visitor.visit_vars(self)

class Var(BaseBox):
    def __init__(self, id, tp):
        self.id = id
        self.tp = tp

    def accept(self, visitor):
        visitor.visit_var(self)


class Instructions(BaseBox):
    def __init__(self, instr, instrs):
        self.instr = instr
        self.instrs = instrs

    def accept(self, visitor):
        visitor.visit_instructions(self)

class Instruction(BaseBox):
    def __init__(self, instr):
        self.instr = instr

    def accept(self, visitor):
        visitor.visit_instruction(self)


class Atrib(BaseBox):
    def __init__(self, id, expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)

class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1

    def accept(self, visitor):
        visitor.visit_while(self)

class Print(BaseBox):
    def __init__(self, value):
        self.value = value

    def accept(self, visitor):
        visitor.visit_print(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value

## CALLING A FUNCTIONS IS AN EXPRESSION
class Call(Expr):
    def __init__(self, func_id, arg):
        self.func_id = func_id
        self.arg = arg

class Return(BaseBox):
    def __init__(self, expr):
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_return(self)

## RESTO


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    pass


class Sub(BinaryOp):
    pass


class Mul(BinaryOp):
    pass


class Div(BinaryOp):
    pass

class Pow(BinaryOp):
    pass

In [4]:
from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS', 'OPEN_CURLY', 'CLOSE_CURLY', 'COLON',
    'PLUS', 'MINUS', 'MUL', 'DIV', 'POW', 'INT', 'FLOAT', 'VOID', 'ID','ENDLINE',
    'EQUALS','COMP','IF','ELSE','WHILE', 'MAIN', 'PRINT', 'STRING', 'RETURN'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV']),
        ('left', ['POW'])
    ]
)

@pg.production('main : MAIN OPEN_CURLY vars instructions CLOSE_CURLY')
def prog(p):
    return Main(p[2],p[3], None)

@pg.production('main : MAIN OPEN_CURLY vars instructions CLOSE_CURLY functions')
def prog(p):
    return Main(p[2], p[3], p[5])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vars : var')
def vars(p):
    return Vars(p[0],None)

@pg.production('vars : var vars')
def vars(p):
    return Vars(p[0],p[1])

@pg.production('var : FLOAT COLON ID ENDLINE')
def var_float(p):
    return Var(p[2].getstr(), "float")

@pg.production('var : INT COLON ID ENDLINE')
def var_int(p):
    return Var(p[2].getstr(), "int")

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('instructions : instruction')
def instruction_instruction(p):
    return Instructions(p[0],None)

@pg.production('instructions : instruction instructions')
def instruction_instructions(p):
    return Instructions(p[0],p[1])

@pg.production('instruction : ID EQUALS expression ENDLINE')
def instruction_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('instruction : PRINT OPEN_PARENS STRING CLOSE_PARENS ENDLINE')
def print_instruction(p):
    return Print(p[2].getstr())

@pg.production('instruction : PRINT OPEN_PARENS expression CLOSE_PARENS ENDLINE')
def print_instruction(p):
    return Print(p[2])

@pg.production('instruction : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_CURLY instructions CLOSE_CURLY')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[7],None)

@pg.production('instruction : IF OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_CURLY instructions CLOSE_CURLY ELSE OPEN_CURLY instructions CLOSE_CURLY')
def expression_ifelse2(p):
    return IfElse (p[2],p[3],p[4],p[7],p[11])

@pg.production('instruction : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS OPEN_CURLY instructions CLOSE_CURLY')
def instruction_while(p):
    return While (p[2],p[3],p[4],p[7])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    # Verifica se é float ou int para fazer o casting
    value = p[0].getstr()

    if "." in value:
        value = float(value)
    else:
        value = int(value)

    return Number(value)

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
@pg.production('expression : expression POW expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    elif p[1].gettokentype() == 'POW':
        return Pow(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

### FUNCTIONS

@pg.production('functions : function functions')
def functions(p):
    return Funcs(p[0], p[1])

@pg.production('functions : function')
def function(p):
    return Funcs(p[0], None)

@pg.production('function : ret_type ID OPEN_PARENS arg CLOSE_PARENS OPEN_CURLY vars instructions return CLOSE_CURLY')
def func_def(p):
    return Func(p[0], p[1].getstr(), p[3], p[6], p[7], p[8])

@pg.production('arg : INT COLON ID')
def arg_int(p):
    return Var(p[2].getstr(), "int")

@pg.production('arg : FLOAT COLON ID')
def arg_float(p):
    return Var(p[2].getstr(), "float")

@pg.production('function : ret_type ID OPEN_PARENS CLOSE_PARENS OPEN_CURLY vars instructions return CLOSE_CURLY')
def func_def(p):
    return Func(p[0], p[1].getstr(), None, p[5], p[6], p[7])

@pg.production('expression : ID OPEN_PARENS ID CLOSE_PARENS')
def expr_call(p):
    return Call(p[0].getstr(), p[2].getstr())

@pg.production('expression : ID OPEN_PARENS CLOSE_PARENS')
def expr_call(p):
    return Call(p[0].getstr(), None)

@pg.production('instruction : ID OPEN_PARENS ID CLOSE_PARENS ENDLINE')
def expr_call(p):
    return Call(p[0].getstr(), p[2].getstr())

@pg.production('instruction : ID OPEN_PARENS CLOSE_PARENS ENDLINE')
def expr_call(p):
    return Call(p[0].getstr(), None)

@pg.production('return : RETURN expression ENDLINE')
def expr_return(p):
    return Return(p[1])

@pg.production('return : RETURN ENDLINE')
def expr_return_empty(p):
    return Return(None)

@pg.production('ret_type : INT')
def ret_int(p):
    return 'int'

@pg.production('ret_type : FLOAT')
def ret_int(p):
    return 'float'

@pg.production('ret_type : VOID')
def ret_void(p):
    return 'void'


parser = pg.build()

In [5]:
class Visitor(object):
    pass

class SymbolTable(Visitor):
    def __init__(self):
        self.ST = {}

    def visit_main(self, main):
        main.vars.accept(self)
        if main.funcs != None:
            main.funcs.accept(self)

    def visit_funcs(self, f):
        f.func.accept(self)
        if f.funcs != None:
            f.funcs.accept(self)

    def visit_func(self, f):
        f.vars.accept(self)
        if f.arg != None:
            f.arg.accept(self)

    def visit_vars(self, v):
        v.var.accept(self)
        if v.vars != None:
            v.vars.accept(self)

    def visit_var(self, v):
        self.ST[v.id] = v.tp

In [6]:
program = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
}"""
symbol = SymbolTable()
arvore=parser.parse(lexer.lex(program))
arvore.accept(symbol)
print(symbol.ST)

{'a': 'int', 'b': 'float'}


In [7]:
program2 = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
    hello();
}

void hello() {
  int: x;
  x = 1;
  print("hello");
  print(x);
  return;
}"""
symbol = SymbolTable()
arvore=parser.parse(lexer.lex(program2))
arvore.accept(symbol)
print(symbol.ST)

{'a': 'int', 'b': 'float', 'x': 'int'}


In [8]:
program = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
    a = hello(b);
    print(a);
}
int hello(int: x) {
  int: y;
  y = x + 1;
  print("hello");
  print(y);
  return y;
}"""
symbol = SymbolTable()
arvore=parser.parse(lexer.lex(program))
arvore.accept(symbol)
print(symbol.ST)

{'a': 'int', 'b': 'float', 'y': 'int', 'x': 'int'}


In [9]:
class FuncTable(Visitor):
    def __init__(self):
        self.FT = {}

    def visit_main(self, main):
        if main.funcs != None:
            main.funcs.accept(self)

    def visit_funcs(self, f):
        f.func.accept(self)
        if f.funcs != None:
            f.funcs.accept(self)

    def visit_func(self, f):
        self.FT[f.id] = f

In [10]:
program = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
    hello(a);
}
int hello(int: x) {
  int: y;
  y = x + 1;
  print("hello");
  print(y);
  return y;
}"""
symbol = SymbolTable()
func = FuncTable()
arvore=parser.parse(lexer.lex(program))
arvore.accept(symbol)
print(symbol.ST)
arvore.accept(func)
print(func.FT)

{'a': 'int', 'b': 'float', 'y': 'int', 'x': 'int'}
{'hello': <__main__.Func object at 0x0000016A1D0A6050>}


In [11]:
class Decorator(Visitor):
    def __init__(self, ST, FT):
        self.ST = ST
        self.FT = FT

    def visit_main(self, p):
        p.instrs.accept(self)
        if p.funcs != None:
            p.funcs.accept(self)

    def visit_funcs(self, f):
        f.func.accept(self)
        if f.funcs != None:
            f.funcs.accept(self)

    def visit_func(self, f):
        f.instrs.accept(self)
        f.ret.accept(self)

    def visit_instructions(self, i):
        i.instr.accept(self)
        if i.instrs!=None:
            i.instrs.accept(self)

    def visit_instruction(self, i):
        i.instr.accept(self)

    def visit_atrib(self, i):
        if i.id in self.ST:
          i.decor_type = self.ST[i.id]
        else:
          raise AssertionError('id not declared')
        i.expr.accept(self)

    def visit_print(self, i):
        if isinstance(i.value, Expr):
            i.value.accept(self)

    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
          i.ie2.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)

    def visit_return(self, r):
        if r.expr != None:
          r.expr.accept(self)
          r.decor_type = r.expr.decor_type
        else:
          r.decor_type = 'void'

    def visit_call(self, c):
        if not c.func_id in self.FT:
          raise AssertionError('function not declared')
        c.decor_type = self.FT[c.func_id].ret_type

    def visit_id(self, i):
        if i.value in self.ST:
          i.decor_type = self.ST[i.value]
        else:
          raise AssertionError('id not declared')

    def visit_number(self, i):
        if "." in str(i.value):
          i.decor_type='float'
        else:
          i.decor_type='int'


    # Segue as regras de operação
    # INT op INT = INT
    # INT op FLOAT = FLOAT
    # FLOAT op FLOAT = FLOAT
    def visit_add(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="float" or a.right.decor_type=="float":
          a.decor_type="float"
        else:
          a.decor_type="int"


    def visit_sub(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="float" or a.right.decor_type=="float":
          a.decor_type="float"
        else:
          a.decor_type="int"

    def visit_mul(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type =="float" or a.right.decor_type=="float":
          a.decor_type="float"
        else:
          a.decor_type="int"

    def visit_div(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="float" or a.right.decor_type=="float":
          a.decor_type="float"
        else:
          a.decor_type="int"

    def visit_pow(self, a):
        a.left.accept(self)
        a.right.accept(self)
        if a.left.decor_type=="float" or a.right.decor_type=="float":
            a.decor_type="float"
        else:
            a.decor_type="int"

In [12]:
class TypeVerifier(Visitor):
    def __init__(self, ST, FT):
        self.ST = ST
        self.FT = FT

    def visit_main(self, i):
        i.instrs.accept(self)
        if i.funcs != None:
          i.funcs.accept(self)

    def visit_funcs(self, f):
        f.func.accept(self)
        if f.funcs != None:
          f.funcs.accept(self)

    def visit_func(self, f):
        f.instrs.accept(self)
        if f.ret_type != f.ret.decor_type:
            raise AssertionError(f'cannot return expression of type {f.ret.decor_type} if the return type of the function is {f.ret_type}')

    def visit_instructions(self, d):
        d.instr.accept(self)
        if d.instrs!=None:
            d.instrs.accept(self)

    def visit_instruction(self, d):
        d.instr.accept(self)

    def visit_call(self, c):
        # O argumento do call tem que ter o mesmo tipo do argumento da função
        if c.arg != None and self.ST[c.arg] != self.ST[self.FT[c.func_id].arg.id]:
            raise AssertionError(f'cannot pass variable of type {self.ST[c.arg]} to function that accepts type {self.FT[c.func_id].arg.tp}')

    def visit_return(self, r):
        pass ## todas as checagens do return acontecem na checagem da função ou na checagem da atribuição

    def visit_atrib(self, i):
        if isinstance(i.expr, Call):
            i.expr.accept(self)
            if i.decor_type != self.FT[i.expr.func_id].ret.decor_type:
              raise AssertionError(f'return type {self.FT[i.expr.func_id].ret.decor_type} cannot be assigned to variable of type {i.decor_type}')
        if i.decor_type != i.expr.decor_type:
            raise AssertionError(f'expression type {i.expr.decor_type} cannot be assigned to variable of type {i.decor_type}')

    def visit_print(self, i):
        pass

    def visit_ifelse(self, i):
        pass

    def visit_while(self, i):
        pass

In [13]:
program = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
}"""

In [14]:
symbol = SymbolTable()
func = FuncTable()
arvore=parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))

In [15]:
program2 = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
    hello();
}
void hello() {
  int: x;
  x = 1;
  print("hello");
  print(x);
  return;
}"""

In [16]:
symbol = SymbolTable()
func = FuncTable()
arvore=parser.parse(lexer.lex(program2))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))

In [17]:
program3 = """ main {
    int: a;
    float: b;
    a = 2;
    b = 3.5;
    if (a != b) {
        print(a + b);
    }
    a = hello();
}
int hello() {
  int: x;
  x = 1;
  print("hello");
  print(x);
  return x;
}"""

In [18]:
symbol = SymbolTable()
func = FuncTable()
arvore=parser.parse(lexer.lex(program3))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))

In [19]:
class Executor(Visitor):
    def __init__(self, FT):
        self.result_table = {}
        self.curr_res = []
        self.FT = FT

    def visit_main(self, main):
        main.vars.accept(self)
        main.instrs.accept(self)

    def visit_func(self, f):
        # não precisamos do visit_funcs pq só visito a função se ela for chamada
        f.vars.accept(self)
        f.instrs.accept(self)
        f.ret.accept(self)

    def visit_vars(self, vars):
        vars.var.accept(self)
        if vars.vars:
            vars.vars.accept(self)

    def visit_var(self, var):
        self.result_table[var.id] = None

    def visit_instructions(self, instructions):
        instructions.instr.accept(self)
        if instructions.instrs:
            instructions.instrs.accept(self)

    def visit_instruction(self, instruction):
        instruction.instr.accept(self)

    def visit_atrib(self, atrib):
        if isinstance(atrib.expr, Call):
          c = atrib.expr
          if self.FT[c.func_id].arg != None:
            self.result_table[self.FT[c.func_id].arg.id] = self.result_table[c.arg]
          self.FT[c.func_id].accept(self)
        else:
          atrib.expr.accept(self)

        value = self.curr_res.pop()
        self.result_table[atrib.id] = value

    def visit_print(self, p):
        if isinstance(p.value, Expr):
            p.value.accept(self)
            value = self.curr_res.pop()
            print(value)
        else:
            print(p.value)

    def visit_ifelse(self, ifelse):
        ifelse.expr1.accept(self)
        ifelse.expr2.accept(self)

        e2 = self.curr_res.pop()
        e1 = self.curr_res.pop()

        condition_value = self.evaluate_comp(ifelse.comp, e1, e2)

        if condition_value:
            ifelse.ie1.accept(self)
        elif ifelse.ie2:
            ifelse.ie2.accept(self)

    def visit_while(self, w):
        w.expr1.accept(self)
        w.expr2.accept(self)

        e2 = self.curr_res.pop()
        e1 = self.curr_res.pop()

        condition_value = self.evaluate_comp(w.comp, e1, e2)

        while condition_value:
            w.ie1.accept(self)
            w.expr1.accept(self)
            w.expr2.accept(self)

            e2 = self.curr_res.pop()
            e1 = self.curr_res.pop()

            condition_value = self.evaluate_comp(w.comp, e1, e2)

    def evaluate_comp(self, comp, left, right):
        comp = comp.getstr()
        if comp == '==':
            return left == right
        elif comp == '!=':
            return left != right
        elif comp == '>=':
            return left >= right
        elif comp == '>':
            return left > right
        elif comp == '<=':
            return left <= right
        elif comp == '<':
            return left < right
        else:
            raise AssertionError('Oops, this should not be possible!')

    def visit_call(self, c):
        if self.FT[c.func_id].arg != None:
          self.result_table[self.FT[c.func_id].arg.id] = self.result_table[c.arg]
        self.FT[c.func_id].accept(self)
        # se o call é instrução, o retorno não é util
        if (len(self.curr_res) > 0):
          self.curr_res.pop()

    def visit_return(self, r):
        if r.expr != None:
          r.expr.accept(self)

    def visit_number(self, number):
        self.curr_res.append(number.value)

    def visit_id(self, id):
        self.curr_res.append(self.result_table[id.value])

    def visit_add(self, add):
        add.left.accept(self)
        add.right.accept(self)
        right_value = self.curr_res.pop()
        left_value = self.curr_res.pop()
        self.curr_res.append(left_value + right_value)

    def visit_sub(self, sub):
        sub.left.accept(self)
        sub.right.accept(self)
        right_value = self.curr_res.pop()
        left_value = self.curr_res.pop()
        self.curr_res.append(left_value - right_value)

    def visit_mul(self, mul):
        mul.left.accept(self)
        mul.right.accept(self)
        right_value = self.curr_res.pop()
        left_value = self.curr_res.pop()
        self.curr_res.append(left_value * right_value)

    def visit_div(self, div):
        div.left.accept(self)
        div.right.accept(self)
        right_value = self.curr_res.pop()
        left_value = self.curr_res.pop()
        self.curr_res.append(left_value / right_value)

    def visit_pow(self, pow):
        pow.left.accept(self)
        pow.right.accept(self)
        right_value = self.curr_res.pop()
        left_value = self.curr_res.pop()
        self.curr_res.append(left_value ** right_value)


In [20]:
symbol = SymbolTable()
func = FuncTable()
exec = Executor(func.FT)

arvore = parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))
arvore.accept(exec)

print(exec.result_table)

5.5
{'a': 2, 'b': 3.5}


In [21]:
program = """ main {
    int: a;
    float: b;
    a = 2;
    b = 5.0;
    if (a != b) {
        while (a < b) {
            a = a ^ 2;
        }
    }
}"""

symbol = SymbolTable()
func = FuncTable()
exec = Executor(func.FT)

arvore = parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))
arvore.accept(exec)

print(exec.result_table)

{'a': 16, 'b': 5.0}


In [22]:
program = """ main {
    int: a;
    int: b;
    a = 2;
    b = 3;
    if (a != b) {
        print(a + b);
    }
    a = hello(b);
    print(a);
}

int hello(int: x) {
  int: y;
  y = x + 1;
  print("hello");
  print(y);
  return y;
}"""


symbol = SymbolTable()
func = FuncTable()
exec = Executor(func.FT)

arvore = parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))
arvore.accept(exec)

print(exec.result_table)

5
"hello"
4
4
{'a': 4, 'b': 3, 'x': 3, 'y': 4}


In [23]:
program = """ main {
    float: x;

    x = 5.0;

    while (x > 1) {
        x = sqrt(x);
    }

    print(x);
}

float sqrt(float: y) {
  float: aux;
  print(y);
  aux = y ^ 0.5;
  return aux;
}"""

symbol = SymbolTable()
func = FuncTable()
exec = Executor(func.FT)

arvore = parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))
arvore.accept(exec)

print(exec.result_table)

5.0
2.23606797749979
1.4953487812212205
1.2228445449938519
1.1058230170302352
1.0515811984959769
1.0254663322098765
1.0126531154397722
1.006306670672401
1.0031483791904372
1.0015729525054264
1.0007861672232619
1.0003930063846218
1.0001964838893516
1.00009823711941
1.0000491173534478
1.000024558375167
1.0000122791121953
1.0000061395372506
1.0000030697639135
1.000001534880779
1.000000767440095
1.0000003837199738
1.0000001918599686
1.0000000959299797
1.0000000479649886
1.000000023982494
1.0000000119912469
1.0000000059956233
1.0000000029978116
1.0000000014989057
1.0000000007494527
1.0000000003747263
1.000000000187363
1.0000000000936815
1.0000000000468408
1.0000000000234204
1.0000000000117102
1.000000000005855
1.0000000000029274
1.0000000000014637
1.0000000000007319
1.000000000000366
1.000000000000183
1.0000000000000915
1.0000000000000457
1.0000000000000229
1.0000000000000113
1.0000000000000056
1.0000000000000027
1.0000000000000013
1.0000000000000007
1.0000000000000002
1.0
{'x': 1.0, 'y': 1

In [25]:
with open('in01.kbr', 'r') as f:
    program = f.read()

symbol = SymbolTable()
func = FuncTable()
exec = Executor(func.FT)

arvore = parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(func)
arvore.accept(Decorator(symbol.ST, func.FT))
arvore.accept(TypeVerifier(symbol.ST, func.FT))
arvore.accept(exec)

print(exec.result_table)

"Calculating the factorial of:"
7
"Partial result:"
7
"Partial result:"
42
"Partial result:"
210
"Partial result:"
840
"Partial result:"
2520
"Partial result:"
5040
{'a': 1, 'res': 5040}


In [ ]:
with open('in02.kbr', 'r') as f:
    program = f.read()

exec = Executor()
symbol = SymbolTable()

arvore = parser.parse(lexer.lex(program))
arvore.accept(symbol)
arvore.accept(Decorator(symbol.ST))
arvore.accept(TypeVerifier())
arvore.accept(exec)

print(exec.result_table)

22
21
20
19
18
17
16
"Acabou!"
{'a': 0, 'b': 15}


Ideias:

* Fazer executável